# Rede Neural para predição da qualidade do sono

Antes de executar este notebook jupyter, primeiro execute o comando para instalar as dependências necessárias:

`
pip install -r requirements.txt
`

In [ ]:
import kagglehub
import os
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

## Dataset

O dataset utilizado pode ser encontrado no [Kaggle](https://www.kaggle.com/datasets/uom190346a/sleep-and-health-metrics)

In [14]:
KAGGLE_DATASET = "uom190346a/sleep-and-health-metrics"
LABEL_COLUMN = "Sleep_Quality_Score"
SEED = 42
TEST_SIZE = 0.2

## Randomização consistente

A utilização de uma SEED fixa, permite a reprodução consistente dos mesmo resultados entre execuções

Importante para evitar resultados inesperados em execuções diferentes

In [15]:
import numpy as np
import random
import tensorflow as tf

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.config.experimental.enable_op_determinism()

In [16]:
def download_dataset(dataset: str) -> str:
    dataset_directory = kagglehub.dataset_download(dataset)

    dataset_file = os.listdir(dataset_directory)[0]

    dataset_file_path = os.path.join(dataset_directory, dataset_file)

    return dataset_file_path

In [17]:
def load_dataset(file_path: str):
    dataset = pd.read_csv(file_path)

    return dataset

## Construção/Arquitetura do modelo

A rede neural consiste de 5 camadas:
* Camada de entrada: 8 neurônios/entradas
* Camada 1: 64 nurônios, ReLU e 20% de dropout
* Camada 2: 64 nurônios, ReLU e 20% de dropout
* Camada 3: 32 nurônios, ReLU
* Camada 4: 16 nurônios, ReLU
* Camada 5: 8 nurônios, ReLU
* Camada de saída: 1 neurônio, Linear

Também há a inclusão de um regularizador "l2" para penalizar pesos muito grandes na rede, em uma tentativa de evitar overfitting dos valores

In [18]:
def build_model(dataset: pd.DataFrame) -> keras.Model:
    input_shape = (dataset.shape[1],)

    l2_reg = regularizers.l2(1e-4)

    model = keras.Sequential(
        [
            layers.Input(shape=input_shape),
            layers.Dense(64, activation="relu", kernel_regularizer=l2_reg),
            layers.Dropout(0.2),
            layers.Dense(64, activation="relu", kernel_regularizer=l2_reg),
            layers.Dropout(0.2),
            layers.Dense(32, activation="relu", kernel_regularizer=l2_reg),
            layers.Dense(16, activation="relu", kernel_regularizer=l2_reg),
            layers.Dense(8, activation="relu", kernel_regularizer=l2_reg),
            layers.Dense(1, activation="linear"),
        ]
    )

    return model

In [19]:
dataset_directory = download_dataset(KAGGLE_DATASET)

dataset = load_dataset(dataset_directory)

In [20]:
X = dataset.drop(columns=[LABEL_COLUMN])

Y = dataset[LABEL_COLUMN]

## Separação do dataset

Os dados do dataset são separados em 80% treinamento e 20% teste

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=TEST_SIZE, random_state=SEED
)

## Métricas do modelo

O modelo usa como métrica de loss, o quadrado da média de erros

Onde ao invés de penalizar os error linearmente, os erros são penalizados muito mais quanto maior forem

In [22]:
model = build_model(X_train)

model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=["mean_absolute_error"],
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,489 (29.25 KB)

 Trainable params: 7,489 (29.25 KB)

 Non-trainable params: 0 (0.00 B)

## Treinamento

O treinamento usa o early stop para parar o treinamento antes do seu fim planejado, caso a rede se estagne em seu valor de loss, a fim de evitar overfitting e economizar recursos de hardware quando não mais crescimento real da rede

In [23]:
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=20,
    restore_best_weights=True
)

history = model.fit(
    X_train,
    Y_train,
    validation_split=0.2,
    epochs=500,
    batch_size=32,
    callbacks=[early_stop]
)

Epoch 1/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 25.1761 - mean_absolute_error: 3.7243 - val_loss: 8.7201 - val_mean_absolute_error: 2.0652
Epoch 2/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9.7800 - mean_absolute_error: 2.4196 - val_loss: 4.3304 - val_mean_absolute_error: 1.4595
Epoch 3/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6.6719 - mean_absolute_error: 2.0078 - val_loss: 2.4153 - val_mean_absolute_error: 1.1554
Epoch 4/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.0481 - mean_absolute_error: 1.6949 - val_loss: 1.7490 - val_mean_absolute_error: 1.0570
Epoch 5/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.4929 - mean_absolute_error: 1.4318 - val_loss: 1.4122 - val_mean_absolute_error: 0.9459
Epoch 6/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.0334 - mean_absolute_error: 1.2496 - val_loss: 1.1928 - val_mean_absolute_error: 0.8437
Epoch 7/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.1874 - mean_absolute_error: 1.0482 - val_lo

## Validação

Para validação da rede, podemos observar as métricas de loss, MAE e R2

Loss e MAE indicam a quantidade de erros da rede, quanto mais próximo de zero melhor, valores menores que 5% são considerados muito bons

O R2 compara os valores reais contra os valores preditos para dar um valor de acerto da rede, quanto mais próximo de 1.0 (100%), melhor

In [24]:
test_loss, test_mae = model.evaluate(X_test, Y_test, verbose=0)

Y_pred = model.predict(X_test).squeeze()
r2 = r2_score(Y_test, Y_test)

print(f'Test loss: {test_loss:.4f}')
print(f'Test MAE: {test_mae:.4f}')
print(f'R2: {r2:.4f}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


2025-11-16 18:20:04.619320: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


Test loss: 0.3553
Test MAE: 0.4133
R2: 1.0000
